# RNN

resource : https://colah.github.io/posts/2015-08-Understanding-LSTMs/

1. Load the extrasensory dataset

In [1]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import train_test_split
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error, classification_report
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader, random_split

from tensorflow.keras.callbacks import EarlyStopping

from utilize.data import *
from utilize.transform import *
from utilize.feature_selection import *
from utilize.test import *

Found 60 users data.


In [2]:
def score_function(y_test, y_pred, score = 'BA', W_test = None):

    mcm = []
    for i in range(y_test.shape[1]):
        if W_test is not None:
            cm = confusion_matrix(y_test[:,i].T, y_pred[:,i].T, sample_weight = W_test[:,i].T)
        else:
            cm = confusion_matrix(y_test[:,i].T, y_pred[:,i].T)
        cm = np.expand_dims(cm, axis = 0)
        mcm.append(cm)
    
    mcm = np.concatenate(mcm, axis = 0)
    tn = mcm[:, 0, 0]
    tp = mcm[:, 1, 1]
    fn = mcm[:, 1, 0]
    fp = mcm[:, 0, 1]
    
    sensitivity = tp / (tp + fn)
    specificity = tn / (tn + fp)
    BA = (sensitivity + specificity)/2
    accuracy = (tn + tp)/(tn + tp + fn + fp)

    sensitivity = np.sum(sensitivity)/sensitivity.shape[0]
    specificity = np.sum(specificity)/specificity.shape[0]
    BA = np.sum(BA)/BA.shape[0]
    accuracy = np.sum(accuracy)/accuracy.shape[0]

    if score == 'BA': 
        return BA
    else: 
        raise Exception('score not valid!')

In [3]:
# Load all the data from Extrasenory dataset
X, y, M, user_index, feature_names, label_names = load_all_data(['1155FF54-63D3-4AB2-9863-8385D0BD0A13'])

In [4]:
feature_index = [ 0,   1,   2,   4,   6,   7,   9,  12,  13,  16,  17,  18,  19,
        20,  22,  23,  24,  25,  26,  27,  28,  31,  32,  34,  35,  36,
        37,  38,  39,  40,  41,  42,  43,  44,  45,  46,  49,  52,  53,
        54,  56,  58,  59,  60,  61,  62,  63,  65,  67,  68,  69,  71,
        72,  73,  74,  76,  77,  78,  79,  80,  81,  85,  87,  88,  89,
        90,  91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102,
       103, 104, 105, 106, 107, 108, 109, 110, 111, 115, 116, 119, 120,
       121, 122, 123, 124, 125, 126, 128, 129, 130, 131, 132, 133, 134,
       135, 136, 137, 139, 140, 142, 144, 145, 146, 147, 148, 149, 150,
       151, 153, 154, 156, 158, 159, 160, 161, 162, 163, 164, 165, 166,
       167, 168, 170, 171, 172, 173, 176, 177, 178, 180, 181, 182, 183,
       185, 187, 188, 190, 192, 193, 194, 195, 196, 197, 198, 199, 201,
       202, 203, 204, 205, 206, 207, 208, 209, 213, 214, 215, 216, 217,
       218, 219, 220, 221, 222, 224]
X175 = X[:, :]

In [5]:
time_domain_keywords = ['mean', 'std', 'moment', 'percentile', 'entropy', 'magnitude_stats', '3d']
frequency_domain_keywords = ['spectrum', 'log_energy_band', 'spectral_entropy', 'magnitude_spectrum']

time_domain_columns = [col for col in feature_names if any(keyword in col for keyword in time_domain_keywords)]
frequency_domain_columns = [col for col in feature_names if any(keyword in col for keyword in frequency_domain_keywords)]

# Display the identified columns
print("Time-Domain Columns:")
for col in time_domain_columns:
    print(col)

print("\nFrequency-Domain Columns:")
for col in frequency_domain_columns:
    print(col)

Time-Domain Columns:
raw_acc:magnitude_stats:mean
raw_acc:magnitude_stats:std
raw_acc:magnitude_stats:moment3
raw_acc:magnitude_stats:moment4
raw_acc:magnitude_stats:percentile25
raw_acc:magnitude_stats:percentile50
raw_acc:magnitude_stats:percentile75
raw_acc:magnitude_stats:value_entropy
raw_acc:magnitude_stats:time_entropy
raw_acc:magnitude_spectrum:spectral_entropy
raw_acc:3d:mean_x
raw_acc:3d:mean_y
raw_acc:3d:mean_z
raw_acc:3d:std_x
raw_acc:3d:std_y
raw_acc:3d:std_z
raw_acc:3d:ro_xy
raw_acc:3d:ro_xz
raw_acc:3d:ro_yz
proc_gyro:magnitude_stats:mean
proc_gyro:magnitude_stats:std
proc_gyro:magnitude_stats:moment3
proc_gyro:magnitude_stats:moment4
proc_gyro:magnitude_stats:percentile25
proc_gyro:magnitude_stats:percentile50
proc_gyro:magnitude_stats:percentile75
proc_gyro:magnitude_stats:value_entropy
proc_gyro:magnitude_stats:time_entropy
proc_gyro:magnitude_spectrum:spectral_entropy
proc_gyro:3d:mean_x
proc_gyro:3d:mean_y
proc_gyro:3d:mean_z
proc_gyro:3d:std_x
proc_gyro:3d:std_y
pro

In [6]:
corr_matrix = np.corrcoef(X175, rowvar=False)
threshold = 0.95
columns_to_remove = np.where(np.abs(corr_matrix) > threshold)
columns_to_remove = [(i, j) for i, j in zip(columns_to_remove[0], columns_to_remove[1]) if i != j]
dedup_columns = list(set([col[0] for col in columns_to_remove]))
X175 = np.delete(X175, dedup_columns, axis=1)
# X175 = X175[:,dedup_columns]

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/numpy/lib/function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/numpy/lib/function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


In [7]:
# Only select body state label
# target_label = ['LYING_DOWN', 'SITTING', 'FIX_walking', 'FIX_running', 'BICYCLING', 'OR_standing']
target_labels = ['LYING_DOWN', 'SITTING', 'FIX_walking']
# Use the last 5 user's data as test set
#test_uuid = list(range(56, 61))

# Fill the Nan with mean value and normalize all the data 
pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy = 'mean')),
    ('std_scaler', StandardScaler())
])

# Transform 
# 1. select target labels 
# 2. tansform feature matrix fill None with mean and do the normalization
# 3. Split train, validation and test set by ratio of 6:2:2
# X_new, y_new, M_new = select_target_labels(X175, y, M, target_label, label_names, drop_all_zero = False)
# X_new = pipeline.fit_transform(X_new, y_new)
# Split the data into training, validation, and test sets
# X_train_val, X_test, y_train_val, y_test, M_train_val, M_test = train_test_split(X_new, y_new, M_new, test_size=0.2, random_state=42)
# X_train, X_val, y_train, y_val, M_train, M_val = train_test_split(X_train_val, y_train_val, M_train_val, test_size=0.25, random_state=42)

In [8]:
# # To swap 0 and 1, simply do 
# y_train_sampleweight = np.abs(1-M_train)

# y_train_sw1D = np.zeros(y_train_sampleweight.shape[0])
# for i in range(len(y_train_sw1D)):
#     y_train_sw1D[i] = np.sum(y_train_sampleweight[i])/M_train.shape[1]

# # To swap 0 and 1, simply do 
# y_test_sampleweight = np.abs(1-M_test)

# y_test_sw1D = np.zeros(y_test_sampleweight.shape[0])
# for i in range(len(y_test_sw1D)):
#     y_test_sw1D[i] = np.sum(y_test_sampleweight[i])/M_test.shape[1]

# RNN model

In [9]:
import warnings
warnings.simplefilter("ignore")

In [10]:
import tensorflow as tf
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, Dropout, LSTM

In [11]:
# print(X_train.shape, X_test.shape)
# print(y_train.shape, y_test.shape)

In [12]:
def run_rnn_model(X_train, X_test, y_train, y_test, M_train, M_test, y_train_sw1D):
    model = Sequential()
    model.add(Dense(1, input_shape=(X_train.shape[1],), activation="relu"))
    model.add(Dropout(0.2))
    model.compile(loss="binary_crossentropy",
                  optimizer='sgd',
                  metrics=['accuracy'])
    callback = EarlyStopping(monitor='loss',
                                patience=3)
    model.fit(X_train, y_train,
                        epochs = 20, 
                        batch_size = 5,
                        validation_data = (X_test, y_test),
                        callbacks=[callback],
                        verbose=0,
                        sample_weight = y_train_sw1D)
    # y_pred = model.predict(X_test)
    # print(classification_report(y_test, (y_pred > 0.5).astype(int)))
    accuracy, precision, recall, f1 = evaluate_model(model, X_test, y_test)
    return accuracy, precision, recall, f1

In [13]:
def run_lstm_model(X_train, X_test, y_train, y_test, M_train, M_test, y_train_sw1D):
    X_train_1 = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
    X_test_1 = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))
    
    model = Sequential()
    model.add(LSTM(1, input_shape=(1, X_train_1.shape[2]), activation="sigmoid"))
    model.add(Dropout(0.2))
    model.compile(loss = "binary_crossentropy",
                optimizer = 'sgd',
                metrics = ['accuracy'])
    callback = EarlyStopping(monitor='loss',
                                patience=3)
    history = model.fit(X_train_1, y_train,
                    epochs = 20, 
                    batch_size = 5,
                    validation_data = (X_test_1, y_test), 
                    callbacks=[callback],
                    verbose=0,
                    sample_weight = y_train_sw1D)
    # Store the performance metric for this split
    accuracy, precision, recall, f1 = evaluate_model(model, X_test_1, y_test)
    return accuracy, precision, recall, f1

In [14]:
def run_random_forest_model(X_train, X_test, y_train, y_test, M_train, M_test, y_train_sw1D):
    rf = RandomForestClassifier(class_weight = 'balanced', n_estimators = 10, min_samples_split = 10)
    rf.fit(X_train, y_train, sample_weight = y_train_sw1D)
    accuracy, precision, recall, f1 = evaluate_model(rf, X_test, y_test)
    return accuracy, precision, recall, f1

In [15]:
def run_svm_model(X_train, X_test, y_train, y_test, M_train, M_test, y_train_sw1D):
    svm = SVC(kernel='linear', random_state=42)
    history = svm.fit(X_train, y_train, sample_weight = y_train_sw1D)
    # y_pred = svm.predict(X_test)
    accuracy, precision, recall, f1 = evaluate_model(svm, X_test, y_test)
    return accuracy, precision, recall, f1

In [16]:

run_model = {
    'rnn': run_rnn_model,
    'lstm': run_lstm_model,
    'random_forest': run_random_forest_model,
    'svm': run_svm_model
}
models = ['rnn', 'lstm', 'random_forest', 'svm']



In [17]:
n_splits = 5
tscv = TimeSeriesSplit(n_splits=n_splits)

In [18]:
def plot_metric(models, values, metric_name):

    # Create the bar graph
    plt.figure(figsize=(10, 6))
    plt.bar(models, values, color='skyblue')

    # Add title and labels
    plt.title(metric_name)
    plt.xlabel('Model')
    plt.ylabel('Values')

    # Show the plot
    plt.show()

In [ ]:

for target_label in target_labels:
    accuracy_scores = {}
    precision_scores = {}
    recall_scores = {}
    f1_scores = {}
    X_new, y_new, M_new = select_target_labels(X175, y, M, target_label, label_names, drop_all_zero = False)
    X_new = pipeline.fit_transform(X_new, y_new)
    for train_index, test_index in tscv.split(X_new, y_new):
        # Split the data into training and test sets based on the current split indices
        X_train, X_test = X_new[train_index], X_new[test_index]
        y_train, y_test = y_new[train_index], y_new[test_index]
        M_train, M_test = M_new[train_index], M_new[test_index]
        # To swap 0 and 1, simply do 
        y_train_sampleweight = np.abs(1-M_train)
        y_train_sw1D = np.zeros(y_train_sampleweight.shape[0])
        for i in range(len(y_train_sw1D)):
            y_train_sw1D[i] = np.sum(y_train_sampleweight[i])/M_train.shape[1]
        
        for model in models:
            accuracy, precision, recall, f1 = run_model[model](X_train,X_test,y_train,y_test,M_train,M_test,y_train_sw1D)
            accuracy_scores[model] = accuracy_scores[model]+[accuracy] if accuracy_scores.get(model) else [accuracy]
            precision_scores[model] = precision_scores[model]+[precision] if precision_scores.get(model) else [precision]
            recall_scores[model] = recall_scores[model]+[recall] if recall_scores.get(model) else [recall]
            f1_scores[model] = f1_scores[model]+[f1] if f1_scores.get(model) else [f1]
            
        # callback = EarlyStopping(monitor='loss',
        #                                         patience=3)
        
        # rnn_model = run_model['rnn'](X_train.shape[1])
        # history = rnn_model.fit(X_train, y_train,
        #                 epochs = 20, 
        #                 batch_size = 5,
        #                 validation_data = (X_test, y_test),
        #                 callbacks=[callback],
        #                 verbose=0,
        #                 sample_weight = y_train_sw1D)
        # # Store the performance metric for this split
        # append_score_to_model('rnn', [np.mean(history.history['val_accuracy'])])
        # y_pred = rnn_model.predict(X_test)
        # # print(classification_report(y_test, (y_pred > 0.5).astype(int)))
        


        # X_train_1 = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
        # X_test_1 = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))
        # lstm_model = run_model['lstm'](X_train_1.shape[2])
        # history = lstm_model.fit(X_train_1, y_train,
        #                 epochs = 20, 
        #                 batch_size = 5,
        #                 validation_data = (X_test_1, y_test), 
        #                 callbacks=[callback],
        #                 verbose=0,
        #                 sample_weight = y_train_sw1D)
        # # Store the performance metric for this split
        # append_score_to_model('lstm', [np.mean(history.history['val_accuracy'])])


        # rf_model = run_model['random_forest']()
        # rf_model.fit(X_train, y_train, sample_weight = y_train_sw1D)
        # append_score_to_model('rf', [rf_model.score(X_test, y_test)])


        # svm_model = run_model['svm']()
        # history = svm_model.fit(X_train, y_train, sample_weight = y_train_sw1D)
        # y_pred = svm_model.predict(X_test)
        # append_score_to_model('svm', [svm_model.score(X_test, y_test)])
        # # print(classification_report(y_test, (y_pred > 0.5).astype(int)))


    # Calculate the mean and standard deviation of the performance metric across all splits
    
    plot_metric(models, [np.mean(val) for val in accuracy_scores.values()], f'Accuracy for {target_label}')
    plot_metric(models, [np.mean(val) for val in precision_scores.values()], f'Precision for {target_label}')
    plot_metric(models, [np.mean(val) for val in recall_scores.values()], f'Recall for {target_label}')
    plot_metric(models, [np.mean(val) for val in f1_scores.values()], f'F1 for {target_label}')
    
# fit network
# model = model_1.fit(X_train, y_train,
#                       epochs = 10, 
#                       batch_size = 16,
#                       validation_data = (X_test, y_test), 
#                       sample_weight = y_train_sw1D)

In [20]:
# Plot training & validation accuracy
#plt.plot(score_rnn)
# font = {'family' : 'DejaVu Sans',
#         'weight' : 'normal',
#         'size'   : 28}
# plt.rc('font', **font)
# fig = plt.figure(figsize = (16, 10))
# score_1 = [0.6319, 0.6612, 0.6808, 0.6891, 0.6992, 0.6936, 0.6981, 0.6944, 0.6956, 0.6979]
# score_2 = [0.6103, 0.7102, 0.7117, 0.7176, 0.7168, 0.7174, 0.7192, 0.7241, 0.7248, 0.7126]
# plt.plot(score_1)
# plt.plot(score_2)
# plt.ylabel('Balanced Accuracy')
# plt.title("Accuarcy Comparison")
# plt.xlabel('Epoch')
# plt.legend(["MLP", "LSTM"], loc = 'bottom right')
# fig.show()

# LSTM model

In [21]:
# [samples, time_steps, features]
X_train_1 = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_test_1 = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))

In [22]:
hidden_size = 1
time_step = 1

model_2 = Sequential()
model_2.add(LSTM(1, input_shape=(time_step, X_train_1.shape[2]), activation="sigmoid"))
model_2.add(Dropout(0.2))
model_2.compile(loss = "binary_crossentropy",
              optimizer = 'sgd',
              metrics = ['accuracy'])

# fit network
model_lstm = model_2.fit(X_train_1, y_train,
                         epochs = 20,
                         batch_size = 1, 
                         validation_data = (X_test_1, y_test),
                         sample_weight = y_train_sw1D)

Epoch 1/20


2238/2238 ━━━━━━━━━━━━━━━━━━━━ 3s 942us/step - accuracy: 0.8648 - loss: 0.4801 - val_accuracy: 0.9351 - val_loss: 0.1644
Epoch 2/20
2238/2238 ━━━━━━━━━━━━━━━━━━━━ 2s 889us/step - accuracy: 0.9339 - loss: 0.3723 - val_accuracy: 0.9485 - val_loss: 0.1390
Epoch 3/20
2238/2238 ━━━━━━━━━━━━━━━━━━━━ 2s 972us/step - accuracy: 0.9398 - loss: 0.3931 - val_accuracy: 0.9530 - val_loss: 0.1316
Epoch 4/20
2238/2238 ━━━━━━━━━━━━━━━━━━━━ 2s 990us/step - accuracy: 0.9534 - loss: 0.2080 - val_accuracy: 0.9597 - val_loss: 0.1272
Epoch 5/20
2238/2238 ━━━━━━━━━━━━━━━━━━━━ 2s 908us/step - accuracy: 0.9534 - loss: 0.3023 - val_accuracy: 0.9620 - val_loss: 0.1268
Epoch 6/20
2238/2238 ━━━━━━━━━━━━━━━━━━━━ 2s 841us/step - accuracy: 0.9422 - loss: 0.3397 - val_accuracy: 0.9642 - val_loss: 0.1199
Epoch 7/20
2238/2238 ━━━━━━━━━━━━━━━━━━━━ 2s 855us/step - accuracy: 0.9494 - loss: 0.3146 - val_accuracy: 0.9664 - val_loss: 0.1151
Epoch 8/20
2238/2238 ━━━━━━━━━━━━━━━━━━━━ 2s 842us/step - accuracy: 0.9541 - loss: 0.22

In [ ]:
plt.plot(scores['rnn'], "RNN")
plt.plot(model_lstm.history['val_acc'], "RNN with LSTM")
plt.title("Accuarcy Comparison")
plt.ylabel('Balanced Accuracy')
plt.xlabel('Epoch')
plt.legend(['RNN', 'RNN with LSTM'], loc = 'upper left')
plt.show()

NameError: name 'scores' is not defined

In [ ]:
# # Plot training & validation accuracy
# #plt.plot(score_rnn)
# list_lstm = model_lstm.history['val_acc'] + [0.6942]
# list_rnn = model.history["acc"]

# fig = plt.figure(figsize = (16, 10))
# plt.plot(model.history['acc'])
# plt.plot(list[1:])
# plt.ylabel('Balanced Accuracy')
# plt.xlabel('Epoch')
# fig.show()

In [ ]:
# plt.plot(model_rnn.history['val_acc'], "RNN")
# plt.plot(model_lstm.history['val_acc'], "RNN with LSTM")
# plt.title("Accuarcy Comparison")
# plt.ylabel('Balanced Accuracy')
# plt.xlabel('Epoch')
# plt.legend(['RNN', 'RNN with LSTM'], loc = 'upper left')
# plt.show()